In [15]:
def He3_temps(MAT, T_AMP, dat_lin):        
    T0 = Temp_fun(MAT, T_AMP, dat_lin, (1/thermnt))
    EDT_import = EDT_calculation(MAT, T_AMP, T0)
    EDT_trnsfrm = interp1d(x = EDT_import[5], y = EDT_import[3], 
                        kind = 'linear',
                        fill_value = "extrapolate")
    EDT_z = EDT_trnsfrm(x = dat_lin)
    
    return EDT_z

In [27]:
def He3_fxn(grain_conc, EDT_z, ER, dat_lin):
    He3P0 = SLHL_He3 * np.exp(-mu * dat_lin)
    grain_conc = [MT_He_3(He3P0[i], grain_conc[i], x, 1, dx, EDT_z[i], nx) for i in range(len(dat_lin))] 

    '''
    int_z        |   return nodes (depths) which remain after erosion;
    first_fxn    |   100 x 100 mesh, 2d interpolation of grain conc. by depth conc.;
    first_data   |   interpolation function applied over linear nodes;
    second_fxn   |   100 x 100 mesh, 2d interpolation of grain conc from linear nodes at new depth;
    grain_conc   |   second function applied back onto the original log nodes.
    '''        

    first_fxn = interp2d(x = range(0,nx), y = dat_lin, z = np.stack(grain_conc), kind = 'cubic')
    first_data = first_fxn(x = range(0,nx), y = dat_lin)
    second_fxn = interp2d(x = range(0,nx), y = (dat_lin - ER), z = np.stack(grain_conc), kind = 'cubic') 
    grain_conc = second_fxn(x = range(0,nx), y = dat_lin)
    
    return grain_conc

In [1]:
def He3_comp_loop_fxn(total_time, initial_ER):
    dat_lin = np.arange(0,max_depth[0] + 1,1)
    
    %run C:\Users\dennis\Documents\STEIN_links\for_publication\He_3_after_Tremblay.ipynb
    %run C:\Users\dennis\Documents\STEIN_links\for_publication\crank_nicolson_bedrock_thermal_field.ipynb

    conc3He = [[0]*nx] * len(dat_lin)
    He_surf = np.empty((total_time,))

    EDT_z = He3_temps(MAT, T_AMP, dat_lin)
   
    ER = initial_ER
    
    for i in range(total_time):
        conc3He = He3_fxn(conc3He, EDT_z, ER, dat_lin)
        He_surf[i] = np.average([np.average(conc3He[i], weights = shell_mass) for i in range(6)])
    
    print('Comparison He3 loop scenario complete.')
    
    np.savetxt(r'C:\Users\dennis\Documents\STEIN_links\for_publication\model_outputs\constant_erosion_for_stochastic_comparisons\He3_comparison_simulation_outputs\STO_He3_ER' + str(ER) + '_surfmat.csv',
               He_surf,
               delimiter = ',')

    return He_surf